[View in Colaboratory](https://colab.research.google.com/github/ZZANZU/DeeeeeepLearning/blob/master/word2vec_tensorflow_kr.ipynb)

# **출처 : 3분 딥러닝**

https://github.com/golbin/TensorFlow-Tutorials/blob/master/04%20-%20Neural%20Network%20Basic/03%20-%20Word2Vec.py

In [37]:
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from google.colab import files

uploaded = files.upload()

# matplot에서 한글을 표시하기위한 설정, 한글 폰트 위치 추가
font_name = matplotlib.font_manager.FontProperties(fname="nanumsquareregular (4).otf").get_name()

matplotlib.rc('font', family=font_name)

Saving nanumsquareregular.otf to nanumsquareregular (4).otf


In [0]:
# 단어 벡터를 분석해볼 임의의 문장들
sentences = ["나 고양이 좋다",
             "나 강아지 좋다",
             "나 동물 좋다",
             "강아지 고양이 동물",
             "여자친구 고양이 강아지 좋다",
             "고양이 생선 우유 좋다",
             "강아지 생선 싫다 우유 좋다",
             "강아지 고양이 눈 좋다",
             "나 여자친구 좋다",
             "여자친구 나 싫다",
             "여자친구 나 영화 책 음악 좋다",
             "나 게임 만화 애니 좋다",
             "고양이 강아지 싫다",
             "강아지 고양이 좋다"]

In [27]:
# 문장을 전부 합친 후 공백으로 단어들을 나누고 고유한 단어들로 리스트를 만듬
word_sequence = " ".join(sentences).split() # 문장들을 다 합치고(join) 공백 기준으로 쪼개기(split)
print(word_sequence)

word_list = list(set(word_sequence)) # 중복 제거
print(word_list)

# 문자열로 분석하는 것 보다 숫자로 분석하는 것이 훨씬 용이하므로 리스트에서 문자들의 인덱스를 뽑아서 사용하기 위해,
# 이를 표현하기 위한 연관 배열과 단어 리스트에서 단어를 참조 할 수 있는 인덱스 배열을 만듬.
word_dict = {w: i for i, w in enumerate(word_list)}
print(word_dict)

['나', '고양이', '좋다', '나', '강아지', '좋다', '나', '동물', '좋다', '강아지', '고양이', '동물', '여자친구', '고양이', '강아지', '좋다', '고양이', '생선', '우유', '좋다', '강아지', '생선', '싫다', '우유', '좋다', '강아지', '고양이', '눈', '좋다', '나', '여자친구', '좋다', '여자친구', '나', '싫다', '여자친구', '나', '영화', '책', '음악', '좋다', '나', '게임', '만화', '애니', '좋다', '고양이', '강아지', '싫다', '강아지', '고양이', '좋다']
['눈', '여자친구', '고양이', '생선', '싫다', '영화', '나', '우유', '강아지', '책', '좋다', '만화', '애니', '게임', '동물', '음악']
{'눈': 0, '여자친구': 1, '고양이': 2, '생선': 3, '싫다': 4, '영화': 5, '나': 6, '우유': 7, '강아지': 8, '책': 9, '좋다': 10, '만화': 11, '애니': 12, '게임': 13, '동물': 14, '음악': 15}


In [28]:
# 윈도우 사이즈를 1 로 하는 skip-gram 모델을 만듭니다.
# 예) 나 게임 만화 애니 좋다
#   -> ([나, 만화], 게임), ([게임, 애니], 만화), ([만화, 좋다], 애니)
#   -> (게임, 나), (게임, 만화), (만화, 게임), (만화, 애니), (애니, 만화), (애니, 좋다)
skip_grams = []
WINDOW_SIZE = 1

for i in range(1, len(word_sequence) - 1) :
    target = word_dict[word_sequence[i - 1]]
    context = [word_dict[word_sequence[i - WINDOW_SIZE]], word_dict[word_sequence[i + WINDOW_SIZE]]]
    
    for w in context :
        skip_grams.append([target, w])

print(skip_grams)

[[6, 6], [6, 10], [2, 2], [2, 6], [10, 10], [10, 8], [6, 6], [6, 10], [8, 8], [8, 6], [10, 10], [10, 14], [6, 6], [6, 10], [14, 14], [14, 8], [10, 10], [10, 2], [8, 8], [8, 14], [2, 2], [2, 1], [14, 14], [14, 2], [1, 1], [1, 8], [2, 2], [2, 10], [8, 8], [8, 2], [10, 10], [10, 3], [2, 2], [2, 7], [3, 3], [3, 10], [7, 7], [7, 8], [10, 10], [10, 3], [8, 8], [8, 4], [3, 3], [3, 7], [4, 4], [4, 10], [7, 7], [7, 8], [10, 10], [10, 2], [8, 8], [8, 0], [2, 2], [2, 10], [0, 0], [0, 6], [10, 10], [10, 1], [6, 6], [6, 10], [1, 1], [1, 1], [10, 10], [10, 6], [1, 1], [1, 4], [6, 6], [6, 1], [4, 4], [4, 6], [1, 1], [1, 5], [6, 6], [6, 9], [5, 5], [5, 15], [9, 9], [9, 10], [15, 15], [15, 6], [10, 10], [10, 13], [6, 6], [6, 11], [13, 13], [13, 12], [11, 11], [11, 10], [12, 12], [12, 2], [10, 10], [10, 8], [2, 2], [2, 4], [8, 8], [8, 8], [4, 4], [4, 2], [8, 8], [8, 10]]


In [0]:
# skip-gram 데이터에서 무작위로 데이터를 뽑아 입력값과 출력값의 배치 데이터를 생성하는 함수
def random_batch(data, size) :
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(data)), size, replace=False) # replace - 한번 선택한 데이터를 다시 선택할 것인지 결정
    
    for i in random_index:
        random_inputs.append(data[i][0]) # target
        random_labels.append(data[i][1]) # context word
        
    return random_inputs, random_labels

# **옵션 설정**

In [0]:
training_epoch = 300 # 학습 반복 횟수
learning_rate = 0.1 # 학습률
batch_size = 20 # 한 번에 학습할 데이터 크기

embedding_size = 2 # 단어 벡터를 구성할 임베딩 차원의 크기, 여기선 2차원 평면에 그리기 쉽게 2로 설정
num_sampled = 15 # word2vec 모델을 학습시키기 위한 nce_loss 함수에서 사용하기 위한 샘플링 크기, batch_size보다 작아야.
voc_size = len(word_list) # 총 단어의 개수

# **텐서플로우 그래프 정의하기**

In [0]:
inputs = tf.placeholder(tf.int32, shape=[batch_size])
labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

# word2vec의 최종 결과물 벡터(W)
embeddings = tf.Variable(tf.random_uniform([voc_size, embedding_size], -1.0, 1.0))

selected_embed = tf.nn.embedding_lookup(embeddings, inputs)

nce_weights = tf.Variable(tf.random_uniform([voc_size, embedding_size], -1.0, 1.0))
nce_biases = tf.Variable(tf.zeros([voc_size]))

loss= tf.reduce_mean(tf.nn.nce_loss(nce_weights, nce_biases, labels, 
                                    selected_embed, num_sampled, voc_size))

train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss) # GradientOptmizer말고 AdamOptimizer를 썼군

# 학습
with tf.Session() as sess :
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for step in range(1, training_epoch + 1) :
        batch_inputs, batch_labels = random_batch(skip_grams, batch_size)
        
        _, loss_val = sess.run([train_op, loss],
                              feed_dict={inputs: batch_inputs,
                                         labels: batch_labels})
        
        if step % 10 == 0 :
            print("loss at step ", step, ": ", loss_val)
            
    trained_embeddings = embeddings.eval()
    
    
# 결과 확인
for i, label in enumerate(word_list):
    x, y = trained_embeddings[i]
    plt.scatter(x, y)
    plt.annotate(label, xy=(x, y), xytext=(5, 2),
                 textcoords='offset points', ha='right', va='bottom')

plt.show()